In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Day 22: Romberg Integration

In the previous notebook we developed, implemented, and utilized the *Trapezoidal Method* for approximating $\displaystyle{\int_{a}^{b}{f\left(x\right)dx}}$. In particular, that method states

$$\int_{a}^{b}{f\left(x\right)dx} = \sum_{i = 0}^{n-1}{\frac{f\left(x_i\right) + f\left(x_{i+1}\right)}{2}h}$$

where $\displaystyle{h = \frac{b - a}{n}}$ is the width of each sub-interval.

*Romberg Integration* combines the *Trapezoidal Method* with *Richardson Extrapolation* for improved efficiency. Before discussing and implementing *Romberg Integration*, let's remind ourselves of the *Trapezoidal Method* and *Richardson Extrapolation*.

### A Reminder on the Trapezoidal Method

When we implemented `trapezoidalMethod()`, we used a recursive approach to approximate the integral. This recursive approach did not require any additional computations, but it did allow for the tracking of differences in approximate areas between consecutive iterations. This gives us an advantage in that we can measure the convergence of the process.

For convenience, our implementation of the recursive *Trapezoidal Method* is below.

In [ ]:
def trapezoid(f, a, b, Iold, k):
  if k == 1:
    Inew = (f(a) + f(b))*(b - a)/2.0
  else:
    n = 2**(k-2)
    h = (b - a)/n
    x = a + h/2.0
    sum = 0.0
    for i in range(n):
      sum = sum + f(x)
      x = x + h

    Inew = (Iold + h*sum)/2.0

  return Inew

def trapezoidMethod(f, a, b, k):
  Iold = (f(a) + f(b))*(b - a)/2.0

  for i in range(2, k + 1):
    Inew = trapezoid(f, a, b, Iold, i)
    diff = Inew - Iold
    #print("The difference in consecutive estimates was ", diff)
    Iold = Inew

  return Inew

**Example:** We can use our `trapezoidMethod()` function to compute the area underneath $\displaystyle{f\left(x\right) = \left|\sin\left(3x\right)\right|e^x}$ along $\left[0, 3\right]$.

In [ ]:
# @title
def f(x):
  return np.abs(np.sin(3*x)*np.exp(x))

x_vals = np.linspace(0, 3, 250)
y_vals = f(x_vals)

k = 4

x_abscissas = np.linspace(0, 3, 2**(k - 1) + 1)
y_abscissas = f(x_abscissas)

plt.figure(figsize = (12, 4))
plt.plot(x_vals, y_vals, color = "purple")
plt.plot(x_abscissas, y_abscissas, color = "red")
plt.scatter(x_abscissas, y_abscissas, color = "red", s = 25)
plt.vlines(x_abscissas, 0, y_abscissas, color = "red", ls = "--")
plt.fill_between(x_abscissas, 0, y_abscissas, color = "red", alpha = 0.2)

plt.grid()
plt.axvline(x = 0, color = "black")
plt.axhline(y = 0, color = "black")
plt.xlabel("x")
plt.ylabel("y")
plt.title("Trapezoidal Method to Estimate f(x) = |sin(3x)|e^x on [0, 3]")
plt.show()

est_area = trapezoidMethod(f, 0, 3, k)

print("The estimated area using " + str(2**(k-1)) + " subintervals is " + str(est_area))

### A Reminder on Richardson Extrapolation

The *Richardson Extrapolation* technique is a method used to eliminate or reduce error in approximations. As a reminder, the method assumes that the error in approximating a quantity $G$ is $\displaystyle{E = ch^p}$ for some constants $c$ and $p$. The constant $p$ is usually obtained by examining the *order* of the *truncation error*. For the majority of our approaches that has been $\mathcal{O}\left(h^2\right)$.

To use the technique, we obtain two estimates for $G$ using different $h$ values (interval widths). It is common to use $h_2 = h_1/2$. In doing this, we obtain

$$G = \frac{2^pg\left(h_1/2\right) - g\left(h_1\right)}{2^p - 1}$$

where $g\left(h_1\right)$ and $g\left(h_1/2\right)$ are the two estimates obtained for $G$.

## Romberg Integration

*Romberg Integration* combines the *Trapezoidal Method* with *Richardson Extrapolation* to improve the efficiency in using the *Trapezoidal Method* to approximate $\displaystyle{\int_{a}^{b}{f\left(x\right)dx}}$.

We begin with the notation $R_{j,1} = I_j$ where $I_j$ represents the approximation for the integral using $2^{j-1}$ subintervals (as we did in the previous notebook and our implementation of the *recursive trapezoidal method*). As mentioned at the end of the previous notebook, the error in this approximation is $E = c_1h^2 + c_2h^4 + \dots$, where $\displaystyle{h = \frac{b-a}{2^{j-1}}}$

We'll start with $R_{1,1} = I_1$ and $R_{2, 1} = I_2$ -- estimates using one and two subintervals, respectively.

\begin{align*}R_{1, 1} &= \left[f\left(a\right) + f\left(b\right)\right]\frac{H}{2}\\
R_{2,1} &= \frac{1}{2}R_{1,1} + f\left(a + \frac{H}{2}\right)\frac{H}{2}
\end{align*}

Using *Richardson Extrapolation*, we have

\begin{align*} \int_{a}^{b}{f\left(x\right)dx} &\approx \frac{2^2R_{2,1} - R_{1,1}}{2^2 - 1}\\
&= \frac{4}{3}R_{2,1} - \frac{1}{3}R_{1,1}
\end{align*}

We then label $\displaystyle{R_{2,2} = \frac{4}{3}R_{2,1} - \frac{1}{3}R_{1,1}}$, the result of using *Richardson Extrapolation* to eliminate the leading error term ($c_1h^2$). We can organize the results into an array of the form:

$$R = \left[\begin{array}{cc} R_{1,1} & \\ R_{2,1} & R_{2,2}\end{array}\right]$$

Next we compute $R_{3,1}$, $R_{3,2}$ and $R_{3,3}$. Note that $R_{3,1} = I_3$ -- the estimate for the integral with $2^{3-1} = 4$ sub-intervals. We can then use *Richardson Extrapolation* to obtain

$$R_{3,2} = \frac{4}{3}R_{3,1} - \frac{1}{3}R_{2,1}$$

resulting in the array

$$R = \left[\begin{array}{ccc} R_{1,1} &  & \\ R_{2,1} & R_{2,2} & \\ R_{3,1} & R_{3, 2} & \end{array}\right]$$

The two elements in the second column are estimates with errors whose leading term is of the form $ch^4$, so we can use *Richardson Extrapolation* to eliminate that leading error term. In doing so, we obtain

\begin{align*} R_{3,3} &= \frac{2^4R_{3,2} - R_{2,2}}{2^4 - 1}\\
&= \frac{16}{15}R_{3,2} - \frac{1}{15}R_{2,2}
\end{align*}

Note that the dominant error term for $R_{3,3}$ is $\mathcal{O}\left(h^6\right)$. The resulting array $R$ is now

$$R = \left[\begin{array}{ccc} R_{1,1} &  & \\ R_{2,1} & R_{2,2} & \\ R_{3,1} & R_{3,2} & R_{3,3}\end{array}\right]$$

We can continue in this fashion, expanding the array $R$ with more accurate approximations of $\displaystyle{\int_{a}^{b}{f\left(x\right)dx}}$ appearing in the lower-right corner. We continue the process until the difference in diagonal elements is sufficiently small (this means that the approximation for the area is converging).

The general form for computing $R_{i,j}$ appears below:

$$R_{i,j} = \frac{4^{j-1}R_{i, j-1} - R_{i-1, j-1}}{4^{j-1} - 1}$$

### Implementing Romberg Integration

The 2-D array we used in the previous subsection is convenient for our own understanding and for "by-hand" computations. A computer, however, can run *Romberg Integration* using just a 1-D array. As a reminder, space- and run-time efficiency are extremely important considerations in numerical analysis. Usually we are encoding routines because we would like to solve lots of problems very quickly -- perhaps to observe change over short time intervals or to inform real-time decisions made by a system.

To carry out *Romberg Integration* with a 1-D array, recall that computing $R_{i,j}$ depends only on $R_{i, j-1}$ and $R_{i-1, j-1}$. For example, $R_{1,1}$ is never used after $R_{2,2}$ is calculated. This means that after computing $R_{2,2}$ we can get away with just the array

$$R' = \left[\begin{array}{c} R_1' = R_{2,2}\\ R_2' = R_{2,1}\end{array}\right]$$

Once we compute $R_{3,2}$, we no longer need $R_{2,1}$, so we can replace $R_2'$ with $R_{3,2}$. Similarly we can replace $R_{2,2}$ with $R_{3,3}$. That is,

$$R' = \left[\begin{array}{c} R_1' = R_{3,3}\\ R_2' = R_{3,2}\\ R_3' = R_{3,1}\end{array}\right]$$

In general, the array $R'$ at the $k^{th}$ round is given by

$$R_j' = \frac{4^{k-j}R_{j+1}' - R_j'}{4^{k-j} - 1}~~\text{for}~~j = k - 1, k - 2, \cdots, 1$$

We are now ready to write our `rombergIntegration()` routine.

In [ ]:
def richardsonExtrapolation(r, k):
  for j in range(k - 1, 0, -1):
    const = 4.0**(k-j)
    r[j] = (const*r[j+1] - r[j])/(const - 1.0)

  return r

def rombergIntegration(f, a, b, tol = 1.0e-6):
  r = np.zeros(21)
  r[1] = trapezoid(f, a, b, Iold = 0.0, k = 1)

  r_old = r[1]

  for k in range(2, len(r)):
    r[k] = trapezoidMethod(f, a, b, k)
    r = richardsonExtrapolation(r, k)
    if abs(r[1] - r_old) < tol*max(abs(r[1]), 1.0):
      return r[1], 2**(k-1)

    r_old = r[1]

  print("Romberg Integration did not converge.")
  return None

**Example 6.6:** Use Romberg Integration to evaluate $\displaystyle{\int_{0}^{\pi}{\sin\left(x\right)dx}}$ to four decimal places.

> *Solution.*

**Example 6.7:** Use Romberg Integration to evaluate $\displaystyle{\int_{0}^{\sqrt{\pi}}{2x^2\cos\left(x^2\right)dx}}$. Compare the result to the answer for ***Example 6.4***. Notice how much more quickly the convergence occurs!

> *Solution.*

***

## Summary

In this notebook we introduced, implemented, and utilized Romberg Integration to observe really strong improvements in efficiency over the Trapezoidal Method for approximating $\displaystyle{\int_{a}^{b}{f\left(x\right)dx}}$. In particular, **Example 6.7** converged at only 64 subintervals with Romberg Integration versus 4,096 required for the classic Trapezoidal Method. This improved efficiency can make a huge difference if we need to solve problems very quickly to help a system make real-time decisions.